In [18]:
import requests
from openai import OpenAI
from dotenv import load_dotenv
import os

In [19]:
# Load environment variables from .env file
load_dotenv()

# API keys
ninja_key = os.getenv("NINJA_API_KEY")
AWS_ACCESS_KEY_ID = os.getenv('aws_access_key_id')
AWS_SECRET_ACCESS_KEY = os.getenv('aws_secret_access_key')

# Fetch Bitcoin API
url = 'https://api.api-ninjas.com/v1/bitcoin'
headers = {'X-Api-Key': ninja_key}
response = requests.get(url, headers=headers)

In [20]:
if response.status_code == 200:
    data = response.json()
    btc_price = float(data['price'])  # ✅ convert to float
    btc_message = f"Bitcoin is currently priced at ${btc_price:.2f}."
    print(btc_message)
else:
    btc_message = "Failed to fetch Bitcoin data."
    print(btc_message)

Bitcoin is currently priced at $84667.53.


In [21]:
# Python Built-Ins:
import os
from typing import Optional
import sys
import json
import time

# External Dependencies:
import boto3
from botocore.config import Config
import botocore

def get_bedrock_client(
    runtime: Optional[bool] = True,
    aws_access_key_id: Optional[str] = None,
    aws_secret_access_key: Optional[str] = None,
    aws_session_token: Optional[str] = None
):
    if runtime:
        service_name = 'bedrock-runtime'
    else:
        service_name = 'bedrock'

    bedrock_runtime = boto3.client(
        service_name=service_name,
        region_name="us-west-2",  # Change to your preferred region
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        #aws_session_token=aws_session_token  # Optional
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_runtime._endpoint)
    return bedrock_runtime

In [22]:
def invoke_model(body, model_id, accept, content_type):
    """
    Invokes Amazon bedrock model to run an inference
    using the input provided in the request body.
    
    Args:
        body (dict): The invokation body to send to bedrock
        model_id (str): the model to query
        accept (str): input accept type
        content_type (str): content type
    Returns:
        Inference response from the model.
    """

    try:
        start_time = time.time()
        response = bedrock_runtime.invoke_model(
            body=json.dumps(body), 
            modelId=model_id, 
            accept=accept, 
            contentType=content_type
        )
        elapsed_time = time.time() - start_time
        print(f"Model invocation took {elapsed_time:.3f} seconds.")

        return response

    except Exception as e:
        print(f"Couldn't invoke {model_id}")
        raise e

In [23]:
import os
import json
import time
import boto3
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# --- Bedrock Client Setup ---
def get_bedrock_client(runtime=True):
    service_name = "bedrock-runtime" if runtime else "bedrock"
    client = boto3.client(service_name, region_name="us-west-2")
    print("✅ Bedrock client created:", client._endpoint)
    return client

# --- Model Invocation Function ---
def invoke_model(body, model_id, accept, content_type):
    try:
        start_time = time.time()
        response = bedrock_runtime.invoke_model(
            body=json.dumps(body),
            modelId=model_id,
            accept=accept,
            contentType=content_type
        )
        print(f"✅ Model invoked in {time.time() - start_time:.2f} seconds")
        return response
    except Exception as e:
        print("❌ Error invoking model:", e)
        raise

# --- Set up client ---
bedrock_runtime = get_bedrock_client()

# --- Claude 3.5-compatible input ---
# If you'd like to try your own prompt, edit this parameter!
if btc_price:
    prompt_data = (
        f"You are a financial advisor focusing on crypto. Bitcoin is currently priced at ${btc_price:.2f}. "
        f"Explain to a new crypto investor whether this is a good time to invest."
    )
else:
    prompt_data = (
        "You are a financial advisor focusing on crypto. Explain to a new crypto investor if BTC would be a good investment."
    )

messages = [{"role": "user", "content": prompt_data}]

body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 250,
    "messages": messages
}

modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"
accept = "application/json"
contentType = "application/json"

# --- Call Claude ---
response = invoke_model(body, modelId, accept, contentType)
response_body = json.loads(response.get("body").read())

print("\n🧠 Claude's Analysis:\n")
print(response_body.get("content")[0].get("text"))

✅ Bedrock client created: bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)
✅ Model invoked in 8.17 seconds

🧠 Claude's Analysis:

As an AI language model, I must emphasize that I cannot provide real-time financial advice or make investment recommendations. The cryptocurrency market is highly volatile and unpredictable, and any investment decisions should be based on thorough research and consultation with a qualified financial advisor.

However, I can provide some general considerations for a new crypto investor:

1. Market volatility: Bitcoin's price is known for significant fluctuations. The current price of $84,667.53 is considerably higher than historical averages, which could indicate a market peak or continued growth.

2. Personal financial situation: Consider your risk tolerance, investment goals, and overall financial health before investing.

3. Diversification: It's generally advisable not to invest more than you can afford to lose and to diversify your invest